# Шкарбаненко Михаил, Б05-907

# Задача 3.1

Выборка: набор предложений на русском языке с указанием частей речи для каждого слова.

Требуется:

* Рассмотреть последовательность частей речи как марковскую модель. Определить оптимальный порядок марковской модели.

* Обучить скрытую марковскую модель по выборке. Оценить точность предсказания частей речи, посчитать энтропию на выборке.
Важно: в целях ускорения эксперимента рекомендуется взять первые 10 МБ текста из выборки.

# Решение

## 1. Подготовительная часть

### Библиотеки

In [1]:
import os
if not os.path.isfile('/content/nerus_lenta.conllu.gz'):
  ! wget https://storage.yandexcloud.net/natasha-nerus/data/nerus_lenta.conllu.gz

--2023-05-09 13:07:58--  https://storage.yandexcloud.net/natasha-nerus/data/nerus_lenta.conllu.gz
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1961465886 (1.8G) [application/octet-stream]
Saving to: ‘nerus_lenta.conllu.gz’

nerus_lenta.conllu. 100%[===================>]   1.83G  22.6MB/s    in 86s     

2023-05-09 13:09:25 (21.9 MB/s) - ‘nerus_lenta.conllu.gz’ saved [1961465886/1961465886]



In [2]:
try:
  import nerus, nltk
except ModuleNotFoundError:
  ! pip install nerus
  ! pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
from numpy import transpose, log, sum as nsum, unique as nunique
from re import compile, Pattern
from sys import getsizeof
from nerus import load_nerus
from nltk.tag import hmm
from collections import Counter
from more_itertools import take
from scipy.stats.distributions import chi2
from seaborn import heatmap
from matplotlib.pyplot import figure
from sys import getsizeof
from sklearn.model_selection import train_test_split

### Данные

In [4]:
dataset = load_nerus('nerus_lenta.conllu.gz')

In [5]:
def get_tokens(dataset, memory_limit):
  tokens = []
  while getsizeof(tokens) < memory_limit:
    for sent in next(dataset).sents:
      for token in sent.tokens:
        word = token.text.lower()
        pos = token.pos
        if compile('^[а-я]+$').match(word):
          tokens.append((word, pos))
  return tokens

In [6]:
tokens = get_tokens(dataset, 12 * 2 ** 20)

In [7]:
len(tokens)

1503860

In [8]:
train_data, test_data = train_test_split(tokens, test_size=0.05, shuffle=False)

## 2. Обучение и анализ модели

In [9]:
vocab_tags = set(tag for _, tag in train_data)
vocab_words = set(word for word, _ in train_data)
trainer = hmm.HiddenMarkovModelTrainer(vocab_tags, vocab_words)
model = trainer.train_supervised([train_data])

In [10]:
import random
def random_sample(list_, size):
  assert size <= len(list_)
  idx1 = random.randrange(0, len(list_) - size + 1)
  idx2 = idx1 + size
  return list_[idx1: idx2]

### Качество модели на трейн датасете

In [11]:
model.test([train_data[0:5000]], verbose=True)

Test: по/ADP социальным/ADJ вопросам/NOUN татьяна/PROPN голикова/PROPN рассказала/VERB в/ADP каких/DET регионах/NOUN россии/PROPN зафиксирована/VERB наиболее/ADV высокая/ADJ смертность/NOUN от/ADP рака/NOUN сообщает/VERB риа/PROPN новости/PROPN по/ADP словам/NOUN голиковой/PROPN чаще/ADV всего/PRON онкологические/ADJ заболевания/NOUN становились/VERB причиной/NOUN смерти/NOUN в/ADP псковской/ADJ тверской/ADJ тульской/ADJ и/CCONJ орловской/ADJ областях/NOUN а/CCONJ также/ADV в/ADP севастополе/PROPN напомнила/VERB что/SCONJ главные/ADJ факторы/NOUN смертности/NOUN в/ADP россии/PROPN рак/NOUN и/CCONJ болезни/NOUN системы/NOUN кровообращения/NOUN в/ADP начале/NOUN года/NOUN стало/VERB известно/ADJ что/SCONJ смертность/NOUN от/ADP онкологических/ADJ заболеваний/NOUN среди/ADP россиян/NOUN снизилась/VERB впервые/ADV за/ADP три/NUM года/NOUN по/ADP данным/NOUN росстата/PROPN в/ADP году/NOUN от/ADP рака/NOUN умерли/VERB тысяч/NOUN человек/NOUN это/PRON на/ADP процента/NOUN меньше/NUM чем/SCONJ

In [12]:
model.test([random_sample(train_data, 1000)], verbose=True)

Test: с/ADP мск/NOUN ноября/NOUN за/ADP введение/NOUN в/ADP ряде/NOUN областей/NOUN страны/NOUN военного/ADJ положения/NOUN ноября/NOUN проголосовали/VERB депутаты/NOUN верховной/ADJ рады/PROPN планировалось/VERB что/SCONJ положение/NOUN начнет/VERB действовать/VERB с/ADP мск/NOUN ноября/NOUN российские/ADJ военные/ADJ инструкторы/NOUN тренируют/VERB армию/NOUN центральноафриканской/ADJ республики/PROPN видеосюжет/NOUN об/ADP этом/PRON опубликовал/VERB телеканал/NOUN на/ADP в/ADP репортаже/NOUN размещенном/VERB ноября/NOUN показано/VERB как/SCONJ проходят/VERB тренировки/NOUN военных/NOUN в/ADP беренго/PROPN где/ADV ранее/ADV находилась/VERB резиденция/NOUN лидера/NOUN цар/PROPN бокассы/PROPN лица/NOUN российских/ADJ инструкторов/NOUN на/ADP кадрах/NOUN скрыты/VERB на/ADP видео/NOUN также/ADV присутствует/VERB советник/NOUN президента/NOUN цар/PROPN по/ADP национальной/ADJ безопасности/NOUN валерий/PROPN захаров/PROPN он/PRON дает/VERB интервью/NOUN в/ADP котором/PRON отрицает/VERB что

Модель неплохо справляется с предсказанием частей речи для сэмпла слов, которые расположены в начале датасета. Если говорить о качестве на рандомном сэмпле, то явно присутствует проблема зацикливания, как можно видеть из второго теста.

### Качество модели на тест датасете

In [13]:
model.test([test_data[0:1000]], verbose=True)

Test: клиентов/NOUN поскольку/SCONJ уже/ADV более/ADV лет/NOUN в/ADP компании/NOUN широко/ADV используется/VERB ручная/ADJ работа/NOUN высококвалифицированных/ADJ французских/ADJ ремесленников/NOUN компания/NOUN размещается/VERB в/ADP том/DET же/PART фабричном/ADJ здании/NOUN которое/PRON было/AUX построено/VERB в/ADP бабушкой/NOUN софи/PROPN грегуар/PROPN я/PRON выросла/VERB на/ADP фабрике/NOUN ею/PRON всегда/ADV руководили/VERB женщины/NOUN три/NUM поколения/NOUN женщин/NOUN бабушка/NOUN которая/PRON основала/VERB компанию/NOUN мама/NOUN которая/PRON начала/VERB сотрудничать/VERB с/ADP крупными/ADJ товаров/NOUN класса/NOUN люкс/NOUN и/CCONJ я/PRON знакомиться/VERB с/ADP бизнесом/NOUN софи/PROPN начала/VERB с/ADP детства/NOUN буквально/ADV росла/VERB на/ADP фабрике/NOUN мама/NOUN приводила/VERB меня/PRON в/ADP цех/NOUN и/CCONJ я/PRON часами/NOUN рылась/VERB в/ADP перчатках/NOUN примеряя/VERB их/PRON вспоминает/VERB софи/PROPN на/ADP вопрос/NOUN о/ADP трендах/NOUN софи/PROPN грегуар/PR

### Точность предсказания по тэгам

In [14]:
tagged_test = model.tag(list(transpose(test_data[0:1000])[0]))
unique, counts = nunique(transpose(tagged_test)[1], return_counts=True)
tagged_dict = dict(zip(unique, counts))
unique, counts = nunique(transpose(test_data)[1], return_counts=True)
test_dict = dict(zip(unique, counts))

Аналогичная проблема на тестовом датасете.

## 3. Оптимальный порядок языковой модели

In [15]:
def ngrams_and_prefix_counts(tokens, n_max):
    # словарь n-грамм и их частот
    ngrams_counts = {}
    # словарь n-граммных префиксов и их частот
    prefix_counts = {}
    
    n = len(tokens)
    for i in range(n_max):
        ngrams_counts[i + 1] = Counter([tuple(tokens[j : j + i + 1]) for j in range(n - i)])
        prefix_counts[i + 1] = Counter([tuple(tokens[j : j + i] + ['*']) for j in range(n - i)])

    return ngrams_counts, prefix_counts

def unigram_probas(ngram_counts):
    p1 = {}
    n = sum(ngram_counts[1].values())
    for w in ngram_counts[1]:
        p1[w] = ngram_counts[1][w] / n
    return p1


def bigram_probas(ngram_counts, prefix_counts):
    p2 = {}
    for w in ngram_counts[2]:
        pre_w = tuple([w[0]] + ['*'])
        p2[u'{1}|{0}'.format(*w)] = ngram_counts[2][w] / prefix_counts[2][pre_w]
    return p2


def trigram_probas(ngram_counts, prefix_counts):
    p3 = {}
    for w in ngram_counts[3]:
        pre_w = w[:2] + tuple(['*'])
        p3[u'{2}|{1},{0}'.format(*w)] = ngram_counts[3][w] / prefix_counts[3][pre_w]
    return p3
  
def random_n_dict_items(d: dict, n: int) -> dict:
  """gets random n items of a sorted dict by values"""
  return [(d[x], x) for x in take(n, d)]

In [16]:
ngram_counts, prefix_counts = ngrams_and_prefix_counts(tokens, 3)

In [17]:
p1 = unigram_probas(ngram_counts)
random_n_dict_items(p1, 15)

[(0.010612690011038262, (('по', 'ADP'),)),
 (1.0639288231617304e-05, (('социальным', 'ADJ'),)),
 (0.00010173819371484048, (('вопросам', 'NOUN'),)),
 (4.654688601332571e-05, (('татьяна', 'PROPN'),)),
 (5.319644115808652e-06, (('голикова', 'PROPN'),)),
 (0.00038035455428031865, (('рассказала', 'VERB'),)),
 (0.05025002327344301, (('в', 'ADP'),)),
 (2.8593087122471508e-05, (('каких', 'DET'),)),
 (0.00011836208157674252, (('регионах', 'NOUN'),)),
 (0.003334751905097549, (('россии', 'PROPN'),)),
 (8.64442168818906e-06, (('зафиксирована', 'VERB'),)),
 (0.00016690383413349647, (('наиболее', 'ADV'),)),
 (1.9948665434282447e-05, (('высокая', 'ADJ'),)),
 (9.309377202665141e-06, (('смертность', 'NOUN'),)),
 (0.0032230393786655672, (('от', 'ADP'),))]

In [18]:
p2 = bigram_probas(ngram_counts, prefix_counts)
random_n_dict_items(p2, 15)

[(0.0004385964912280702, "('социальным', 'ADJ')|('по', 'ADP')"),
 (0.125, "('вопросам', 'NOUN')|('социальным', 'ADJ')"),
 (0.013071895424836602, "('татьяна', 'PROPN')|('вопросам', 'NOUN')"),
 (0.08571428571428572, "('голикова', 'PROPN')|('татьяна', 'PROPN')"),
 (0.125, "('рассказала', 'VERB')|('голикова', 'PROPN')"),
 (0.06818181818181818, "('в', 'ADP')|('рассказала', 'VERB')"),
 (0.00018526115205970702, "('каких', 'DET')|('в', 'ADP')"),
 (0.06976744186046512, "('регионах', 'NOUN')|('каких', 'DET')"),
 (0.14606741573033707, "('россии', 'PROPN')|('регионах', 'NOUN')"),
 (0.00019940179461615153, "('зафиксирована', 'VERB')|('россии', 'PROPN')"),
 (0.07692307692307693, "('наиболее', 'ADV')|('зафиксирована', 'VERB')"),
 (0.00398406374501992, "('высокая', 'ADJ')|('наиболее', 'ADV')"),
 (0.03333333333333333, "('смертность', 'NOUN')|('высокая', 'ADJ')"),
 (0.5, "('от', 'ADP')|('смертность', 'NOUN')"),
 (0.004126263668248401, "('рака', 'NOUN')|('от', 'ADP')")]

In [19]:
p3 = trigram_probas(ngram_counts, prefix_counts)
random_n_dict_items(p3, 10)

[(0.2857142857142857,
  "('вопросам', 'NOUN')|('социальным', 'ADJ'),('по', 'ADP')"),
 (1.0, "('татьяна', 'PROPN')|('вопросам', 'NOUN'),('социальным', 'ADJ')"),
 (1.0, "('голикова', 'PROPN')|('татьяна', 'PROPN'),('вопросам', 'NOUN')"),
 (0.16666666666666666,
  "('рассказала', 'VERB')|('голикова', 'PROPN'),('татьяна', 'PROPN')"),
 (1.0, "('в', 'ADP')|('рассказала', 'VERB'),('голикова', 'PROPN')"),
 (0.02564102564102564, "('каких', 'DET')|('в', 'ADP'),('рассказала', 'VERB')"),
 (0.14285714285714285, "('регионах', 'NOUN')|('каких', 'DET'),('в', 'ADP')"),
 (0.6666666666666666,
  "('россии', 'PROPN')|('регионах', 'NOUN'),('каких', 'DET')"),
 (0.038461538461538464,
  "('зафиксирована', 'VERB')|('россии', 'PROPN'),('регионах', 'NOUN')"),
 (1.0, "('наиболее', 'ADV')|('зафиксирована', 'VERB'),('россии', 'PROPN')")]

In [20]:
def chi2_stat(p_small, p_big, tokens, model_type: str):
    stat_small = []
    stat_big = []
    n = len(tokens)
    for i in range(n - 2):
        if model_type == "3_to_2":
            w = tokens[i : i + 3]
            ngram_big = '{2}|{1},{0}'.format(*w)
            ngram_small = '{1}|{0}'.format(*w)
        elif model_type == "2_to_1":
            w = tokens[i : i + 2]
            ngram_big = '{1}|{0}'.format(*w)
            ngram_small = '{0}'.format(*w)
    return - 2 * nsum(stat_small) + 2 * nsum(stat_big)

In [21]:
stat_32 = chi2_stat(p2, p3, tokens, model_type="3_to_2")
print(f'p-value for 3 -> 2 reduction: {1 - chi2(len(p3) * ((len(p3) - 1) ** 2) - 1).cdf(stat_32):.2f}')

p-value for 3 -> 2 reduction: 1.00


In [22]:
stat_21 = chi2_stat(p1, p2, tokens, model_type="2_to_1")
print(f'p-value for 2 -> reduction: {1 - chi2(len(p2) * ((len(p2) - 1) ** 2) - 1).cdf(stat_21):.2f}')

p-value for 2 -> reduction: 1.00


# Итоги

* Оптимальная языковая модель имеет порядок 1 (использует монограммы).

* Качество марковской модели получилось довольно низким. Присутствует проблема зацикливания.